In [ ]:
import pandas as pd
import numpy as np
import os

parquet_data = [f"data/{file}" for file in os.listdir("data") if file.endswith(".parquet")]
df_list = []
for file in parquet_data:
    df = pd.read_parquet(file)
    df["Ticker"] = file.split("/")[1].replace(".parquet", "")
    df_list.append(df)

df = pd.concat(df_list)

In [ ]:
df["Error"] = df["Close"] - df["Prediction"]
df["SQE"] = df["Error"]**2
df["LogError"] = np.log(df["Close"]) - np.log(df["Prediction"])
df["PercentageError"] = df["Prediction"]/df["Close"] - 1
df.groupby("Ticker")[["SQE", "LogError", "PercentageError"]].mean()

In [6]:
from utils import plot_wealth

wealth = []
tickers = [
    "PETR4.SA",
    "BBAS3.SA",
    "ITUB4.SA",
    "CSMG3.SA",
    "SAPR11.SA",
    "CMIG4.SA",
    "ISAE4.SA",
]

for ticker_idx, ticker in enumerate(tickers):
    money = [1000]
    quotas = [0]
    wealth.append([money[0]])

    ticker_df = df.query(f"Ticker == '{ticker}'")
    close_sequence = ticker_df["Close"].to_numpy()
    prediction_sequence = ticker_df["Prediction"].to_numpy()

    last_buy = float("inf")

    for i in range(prediction_sequence.shape[0] - 1):
        close = close_sequence[i]
        prediction = prediction_sequence[1 + i]
        pct_change_pred = prediction / close - 1
        last_buy_pct_change = close / last_buy - 1
        # print(prediction, close, pct_change_pred, pct_change_pred > 0.01)

        if pct_change_pred > 0.01:
            quotas.append(quotas[i] + money[i] // close)
            money.append(money[i] % close)
            last_buy = close

        elif pct_change_pred < -0.01:
            money.append(money[i] + quotas[i] * close)
            quotas.append(0)

        else:
            money.append(money[i])
            quotas.append(quotas[i])

        wealth[ticker_idx].append(money[i + 1] + quotas[i + 1] * close)
    plot_wealth(
        ticker_df["Date"].to_numpy(),
        [wealth[ticker_idx]],
        ["Wealth"],
        f"{ticker}",
        f"assets/{ticker}_Wealth.png",
    )

In [9]:
general_wealth = np.array(wealth).sum(axis=0)
plot_wealth(
    ticker_df["Date"].to_numpy(),
    [general_wealth],
    ["Wealth"],
    f"Wealth over time",
    f"assets/Wealth.png",
)